## Mindminer — Movie Recommendation UI

This notebook runs a **Tkinter GUI** for the content-based movie recommender.

- **Input:** User enters a movie title (search is **case-insensitive**: "avatar", "Avatar", "AVATAR" all work).
- **Output:** Up to 5 similar movies in a scrollable list, based on tag similarity.
- **Features:** Search / Clear buttons, Enter key to search, status bar, dark theme.
- **Dependencies:** `recommend(movie)` is imported from `src.training.recommender` in the next cell.

In [1]:
# Add project root to path and import recommend from the training module
import sys
from pathlib import Path

cwd = Path.cwd()
if (cwd / "src" / "training" / "recommender.py").exists():
    sys.path.insert(0, str(cwd))
else:
    for _ in range(3):
        cwd = cwd.parent
        if (cwd / "src" / "training" / "recommender.py").exists():
            sys.path.insert(0, str(cwd))
            break
from src.training.recommender import recommend

In [4]:
import tkinter as tk

# ---- Theme ----
BG = "#1a1a2e"
CARD = "#16213e"
ACCENT = "#0f3460"
HIGHLIGHT = "#e94560"
TEXT = "#eaeaea"
TEXT_DIM = "#a0a0a0"
FONT_UI = ("Segoe UI", 11)
FONT_TITLE = ("Segoe UI", 16, "bold")
FONT_RESULT = ("Segoe UI", 11)

def suggest_movies():
    query = entry.get().strip()
    if not query:
        results_list.delete(0, tk.END)
        results_list.insert(tk.END, "Enter a movie title above and press Search or Enter.")
        return
    status_var.set("Searching...")
    root.update_idletasks()
    try:
        suggested = recommend(query)
        results_list.delete(0, tk.END)
        for m in suggested:
            results_list.insert(tk.END, m)
        status_var.set(f"Found {len(suggested)} result(s). Search is case-insensitive.")
    except Exception as e:
        results_list.delete(0, tk.END)
        results_list.insert(tk.END, f"Error: {e}")
        status_var.set("Error occurred.")
    finally:
        root.update_idletasks()

def clear_all():
    entry.delete(0, tk.END)
    results_list.delete(0, tk.END)
    status_var.set("Enter a movie name (case-insensitive).")
    entry.focus_set()

def on_enter(_):
    suggest_movies()

root = tk.Tk()
root.title("Mindminer — Movie Recommender")
root.geometry("520x420")
root.minsize(400, 320)
root.configure(bg=BG)

# Header
header = tk.Frame(root, bg=BG)
header.pack(fill=tk.X, pady=(20, 10))
tk.Label(header, text="Mindminer", font=FONT_TITLE, fg=HIGHLIGHT, bg=BG).pack()
tk.Label(header, text="Content-based movie recommendations", font=("Segoe UI", 9), fg=TEXT_DIM, bg=BG).pack()

# Search card
card = tk.Frame(root, bg=CARD, padx=20, pady=16)
card.pack(fill=tk.X, padx=24, pady=12)

tk.Label(card, text="Movie title", font=FONT_UI, fg=TEXT, bg=CARD).pack(anchor=tk.W)
entry = tk.Entry(card, font=FONT_UI, width=42, bg=ACCENT, fg=TEXT, insertbackground=TEXT,
                 relief=tk.FLAT, highlightthickness=0)
entry.pack(fill=tk.X, pady=(4, 12))
entry.bind("<Return>", on_enter)
entry.focus_set()

btn_frame = tk.Frame(card, bg=CARD)
btn_frame.pack(fill=tk.X)
suggest_btn = tk.Button(btn_frame, text="Search", command=suggest_movies, font=FONT_UI,
                        bg=HIGHLIGHT, fg="white", activebackground="#c73e54", activeforeground="white",
                        relief=tk.FLAT, padx=20, pady=6, cursor="hand2")
suggest_btn.pack(side=tk.LEFT, padx=(0, 8))
clear_btn = tk.Button(btn_frame, text="Clear", command=clear_all, font=FONT_UI,
                      bg=ACCENT, fg=TEXT, relief=tk.FLAT, padx=16, pady=6, cursor="hand2")
clear_btn.pack(side=tk.LEFT)

# Results
tk.Label(root, text="Recommendations", font=FONT_UI, fg=TEXT, bg=BG).pack(anchor=tk.W, padx=24, pady=(8, 4))
results_frame = tk.Frame(root, bg=CARD, padx=12, pady=8)
results_frame.pack(fill=tk.BOTH, expand=True, padx=24, pady=(0, 12))

scrollbar = tk.Scrollbar(results_frame)
scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
results_list = tk.Listbox(results_frame, font=FONT_RESULT, bg=ACCENT, fg=TEXT, selectbackground=HIGHLIGHT,
                         selectforeground="white", relief=tk.FLAT, highlightthickness=0,
                         yscrollcommand=scrollbar.set)
results_list.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
scrollbar.config(command=results_list.yview)

# Status
status_var = tk.StringVar(value="Enter a movie name (case-insensitive).")
status = tk.Label(root, textvariable=status_var, font=("Segoe UI", 9), fg=TEXT_DIM, bg=BG)
status.pack(fill=tk.X, pady=(0, 12))

root.mainloop()